Main Application

In [3]:
#Importing relevant libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk
from tkinter import filedialog, messagebox, Listbox, Tk, Label, Button, ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from datetime import datetime
import os

In [16]:
# Get the directory where the Jupyter Notebook is running
current_directory = os.getcwd()

# Function to validate file names
def validate_file(label, filename):
    return label.lower().replace(" ", "_") in filename.lower()

# Function to handle file selection
def upload_file(label, display_label, file_type):
    file_path = filedialog.askopenfilename()
    if file_path:
        if validate_file(label, file_path):
            display_label.config(text=f"File: {os.path.basename(file_path)}", fg="green")
            if file_type == "USER_LOG":
                global user_log_path
                user_log_path = file_path
            elif file_type == "ACTIVITY_LOG":
                global activity_log_path
                activity_log_path = file_path
            elif file_type == "COMPONENT_CODES":
                global component_codes_path
                component_codes_path = file_path
        else:
            display_label.config(text="Invalid file name", fg="red")
            messagebox.showerror("Invalid File", f"The file does not match the required '{label}' label.")

# Function to open the "Receive Files" window
def open_receive_window():
    root.destroy()  # Destroy the main window when this screen opens
    receive_window = tk.Tk()
    receive_window.title("Upload Files")
    receive_window.geometry("550x350")

    labels = ["USER_LOG", "ACTIVITY_LOG", "COMPONENT_CODES"]
    for i, label in enumerate(labels):
        tk.Label(receive_window, text=label).grid(row=i, column=0, padx=10, pady=10, sticky="w")
        display_label = tk.Label(receive_window, text="No file selected", fg="gray")
        display_label.grid(row=i, column=1, padx=10, pady=10, sticky="w")
        tk.Button(receive_window, text="Browse to Upload", command=lambda l=label, dl=display_label, ft=label: upload_file(l, dl, ft)).grid(row=i, column=2, padx=10, pady=10)

    tk.Button(receive_window, text="Cancel", bg="#949494", fg="black", width=15, command=receive_window.destroy).grid(row=4, column=0, padx=10, pady=20, sticky="e")
    tk.Button(receive_window, text="Process Files", bg="#949494", fg="black", width=15, command=lambda: process_files(receive_window)).grid(row=4, column=2, padx=10, pady=20, sticky="w")
    tk.Label(receive_window, text="Please note, you will need to upload all 3 files in order to continue with processing",
             fg="blue").grid(row=5, column=0, columnspan=3, padx=10, pady=(20, 0), sticky="w")

# Function to manage saved tables
def open_table_manager():
    table_manager = tk.Tk()
    table_manager.title("Table Manager")
    table_manager.geometry("950x400")

    # Define the listbox to display saved JSON files
    listbox = Listbox(table_manager, width=130, height=20)
    listbox.grid(row=1, column=0, columnspan=4, padx=10, pady=10)

    # Populate the listbox with saved JSON files
    json_files = [f for f in os.listdir(current_directory) if f.endswith('.json')]
    listbox.insert(tk.END, *json_files)

    # Buttons in the Table Manager
    tk.Button(table_manager, text="View Merged Table", bg="#949494", width=15, height=1, 
              command=lambda: view_merged_table(listbox)).grid(row=0, column=0, padx=0, pady=0)
    tk.Button(table_manager, text="View Statistics", bg="#949494", width=12, height=1, 
              command=lambda: open_statistics_from_manager(listbox)).grid(row=0, column=1, padx=0, pady=0)    
    tk.Button(table_manager, text="View Pivot Table", bg="#949494", width=14, height=1, 
              command=lambda: open_pivot_table_from_manager(listbox)).grid(row=0, column=2, padx=0, pady=0)
    tk.Button(table_manager, text="View Graphs", bg="#949494", width=10, height=1, 
              command=lambda: open_graphs_from_manager(listbox)).grid(row=0, column=3, padx=0, pady=0)
    tk.Button(table_manager, text="Back to Home Page", bg="#949494", width=15, height=1, 
              command=lambda: [table_manager.destroy(), root.deiconify()]).grid(row=0, column=4, padx=0, pady=0)


def open_statistics_from_manager(listbox):
    selected = listbox.curselection()
    if selected:
        file_to_analyze = listbox.get(selected[0])  # Selected file from listbox
        filepath = os.path.join(current_directory, file_to_analyze)  # Full path to the selected file
        view_statistics(filepath)  # Pass the selected file to view_statistics
    else:
        messagebox.showwarning("No Selection", "Please select a table to analyze.")
        

# Open Pivot table button function
def open_pivot_table_from_manager(listbox):
    selected = listbox.curselection()
    if selected:
        file_to_view = listbox.get(selected[0])  # Selected file from listbox
        filepath = os.path.join(current_directory, file_to_view)  # Full path to the selected file
        view_pivot_table(filepath)  # Pass the selected file to the view_pivot_table function
    else:
        messagebox.showwarning("No Selection", "Please select a table to view the pivot table.")
        

# Open Graphs button function
def open_graphs_from_manager(listbox):
    selected = listbox.curselection()
    if selected:
        file_to_view = listbox.get(selected[0])
        filepath = os.path.join(current_directory, file_to_view)
        view_graphs(filepath)
    else:
        messagebox.showwarning("No Selection", "Please select a table to view the graphs.")


# Function to process files
def process_files(window):
    try:
        if not user_log_path or not activity_log_path or not component_codes_path:
            messagebox.showerror("Error", "Please upload all three files before processing.")
            return

        # Reading the CSVs into dataframes
        component_codes_df = pd.read_csv(component_codes_path)
        activity_log_df = pd.read_csv(activity_log_path)
        user_log_df = pd.read_csv(user_log_path)

        # Step 1: Rename columns
        user_log_df.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)
        activity_log_df.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)

        # Step 2: Add unique identifier column for each file
        user_log_df['Unique_ID'] = user_log_df.index + 1
        activity_log_df['Unique_ID'] = activity_log_df.index + 1

        #Step 3: Merge Activity and User Log based on User_ID
        merged_data = pd.merge(user_log_df, activity_log_df, on='Unique_ID', how='inner')
        merged_data = merged_data.drop(columns=['User_ID_y'])
        merged_data.rename(columns={"User_ID_x": "User_ID"}, inplace=True)
        merged_data['Unique_ID'] = merged_data.index + 1
        merged_data['Date'] = pd.to_datetime(merged_data['Date'], format='%d/%m/%Y %H:%M')

        # Step 4: Remove System and Folder Components and drop records with Null values
        filtered_merged_data = merged_data[~merged_data['Component'].isin(['System', 'Folder'])]
        filtered_merged_data = filtered_merged_data.dropna()

        # Save the merged file with a timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        merged_filename = os.path.join(current_directory, f"merged_data_{timestamp}.json")
        filtered_merged_data.to_json(merged_filename, index=False)

        messagebox.showinfo("Processing Complete", f"Files processed successfully and saved as {merged_filename}!")
        window.destroy()
        open_table_manager()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during processing: {str(e)}")


# Function to view the merged table
def view_merged_table(listbox):
    selected = listbox.curselection()
    if selected:
        file_to_view = listbox.get(selected[0])
        filepath = os.path.join(current_directory, file_to_view)
        try:
            # Load the JSON file into a DataFrame
            df = pd.read_json(filepath)

            # Use only the first 10 rows for display
            df_head = df.head(10)

            # Open a new window to display the data
            table_window = tk.Toplevel()
            table_window.title("View Merged Table")
            table_window.geometry("730x300")

            # Buttons at the top of the window
            tk.Button(table_window, text="Back to Table Manager", bg="#949494", width=20, command=table_window.destroy).grid(row=0, column=0, padx=10, pady=10)
            tk.Button(table_window, text="Back to Home Page", bg="#949494", width=20, command=lambda: [table_window.destroy(), root.deiconify()]).grid(row=0, column=2, padx=10, pady=10)

            # Frame to hold the data
            frame = tk.Frame(table_window, borderwidth=2, relief="groove")
            frame.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

            # Add table
            tree = ttk.Treeview(frame, columns=list(df_head.columns), show="headings", height=10)
            tree.pack(side=tk.LEFT, fill=tk.BOTH)

            # Add columns and rows to the table
            for col in df_head.columns:
                tree.heading(col, text=col)
                tree.column(col, width=100, anchor="center")
            for index, row in df_head.iterrows():
                tree.insert("", tk.END, values=list(row))

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while loading the table: {str(e)}")
    else:
        messagebox.showwarning("No Selection", "Please select a table to view.")

# Function to show the statistics window
def view_statistics(filepath):
    try:
        # Load the selected table
        data = pd.read_json(filepath)  # Use the file selected in the View Table function

        # Create a new window for statistics
        statistics_window = tk.Toplevel()
        statistics_window.title("View Statistics")
        statistics_window.geometry("900x400")  # Extended horizontally and vertically

        # Buttons at the top
        tk.Button(statistics_window, text="Back to Table Manager", bg="#949494", width=20, command=statistics_window.destroy).grid(row=0, column=0, padx=10, pady=10)
        tk.Button(statistics_window, text="Back to Home Page", bg="#949494", width=20, command=lambda: [statistics_window.destroy(), root.deiconify()]).grid(row=0, column=1, padx=10, pady=10)

        # Prepare data for statistics
        data['Date'] = pd.to_datetime(data['Date'])
        data['Month'] = data['Date'].dt.to_period('M')
        selected_components = ['Quiz', 'Lecture', 'Assignment', 'Attendence', 'Survey']

        # Group data for monthly statistics
        stats = data[data['Component'].isin(selected_components)].groupby(['Month', 'Component'])['User_ID'].agg(['mean', 'median', lambda x: x.mode()[0]])
        stats.columns = ['Mean', 'Median', 'Mode']

        # Group data for entire semester statistics
        stats_entire_semester = data[data['Component'].isin(selected_components)].groupby('Component')['User_ID'].agg(['mean', 'median', lambda x: x.mode()[0]])
        stats_entire_semester.columns = ['Mean', 'Median', 'Mode']

        # Panel 1: Statistics by Month
        frame1 = tk.Frame(statistics_window, borderwidth=2, relief="groove", width=500, height=50)
        frame1.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        stats_text1 = tk.Text(frame1, wrap=tk.WORD, height=30, width=70)
        stats_text1.insert(tk.END, "Statistics by Month:\n\n")
        stats_text1.insert(tk.END, stats.to_string())
        stats_text1.pack(fill=tk.BOTH)

        # Panel 2: Statistics for Entire Semester
        frame2 = tk.Frame(statistics_window, borderwidth=2, relief="groove", width=500, height=50)
        frame2.grid(row=1, column=1, padx=10, pady=10, sticky="nsew")

        stats_text2 = tk.Text(frame2, wrap=tk.WORD, height=30, width=70)
        stats_text2.insert(tk.END, "Statistics for Entire Semester:\n\n")
        stats_text2.insert(tk.END, stats_entire_semester.to_string())
        stats_text2.pack(fill=tk.BOTH)

        # Make both frames resize with the window
        statistics_window.grid_rowconfigure(1, weight=1)
        statistics_window.grid_columnconfigure(0, weight=1)
        statistics_window.grid_columnconfigure(1, weight=1)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while generating statistics: {str(e)}")


# Function to display graphs of selected file
def view_graphs(filepath):
    try:
        # Load the selected table
        data = pd.read_json(filepath)

        # Add 'Count' column for aggregation
        data['Count'] = 1

        # Create the pivot table with counts of interactions
        pivot_table = data.pivot_table(
            index='User_ID',
            columns='Component',
            values='Count',
            aggfunc='sum',
            fill_value=0
        )
        # Create a contingency table
        contingency_table = pd.crosstab(data['User_ID'], data['Component'])

        # Count total interactions for specific components
        selected_components = ['Assignment', 'Quiz', 'Lecture', 'Book', 'Project', 'Course']
        component_counts = data[data['Component'].isin(selected_components)].groupby('Component')['Count'].sum()

        # Create a new window for the visualizations
        graph_window = tk.Toplevel()
        graph_window.title("Graphs and Tables")
        graph_window.geometry("1400x700")  # Adjust window size

        # Add a frame with scrollbars
        main_frame = tk.Frame(graph_window)
        main_frame.pack(fill=tk.BOTH, expand=True)

        canvas = tk.Canvas(main_frame)
        scrollbar_y = tk.Scrollbar(main_frame, orient=tk.VERTICAL, command=canvas.yview)
        scrollbar_x = tk.Scrollbar(main_frame, orient=tk.HORIZONTAL, command=canvas.xview)

        scrollable_frame = tk.Frame(canvas)
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

        scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
        scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Left Side: Bar graph
        bar_graph_frame = tk.Frame(scrollable_frame, width=1000, height=700)
        bar_graph_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Generate the bar graph for total interactions
        plt.figure(figsize=(10, 7))
        bars = plt.bar(component_counts.index, component_counts.values, color='skyblue', edgecolor='black')
        plt.title("Total Interactions per Component")
        plt.xlabel("Components")
        plt.ylabel("Total Interactions")
        plt.xticks(rotation=45)

        # Add values on top of the bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}',
                     ha='center', va='bottom', fontsize=10, color='black')

        plt.tight_layout()

        # Embed the bar graph in the Tkinter window
        fig_bar_graph = plt.gcf()  # Get the current figure
        bar_graph_canvas = FigureCanvasTkAgg(fig_bar_graph, master=bar_graph_frame)
        bar_graph_canvas.draw()
        bar_graph_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Right Side: Heatmap
        heatmap_frame = tk.Frame(scrollable_frame, width=1000, height=700)
        heatmap_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Generate the heatmap for the contingency matrix
        plt.figure(figsize=(10, 7))
        sns.heatmap(contingency_table, annot=True, cmap='coolwarm', cbar=True)
        plt.title("Correlation Matrix Heatmap (Components)")
        plt.xlabel("Components")
        plt.ylabel("User ID")
        plt.tight_layout()

        # Embed the heatmap in the Tkinter window
        fig_heatmap = plt.gcf()  # Get the current figure
        heatmap_canvas = FigureCanvasTkAgg(fig_heatmap, master=heatmap_frame)
        heatmap_canvas.draw()
        heatmap_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Back to Table Manager button
        tk.Button(scrollable_frame, text="Back to Table Manager", bg="#87CEEB", width=20, command=graph_window.destroy).pack(pady=10)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while generating the graphs: {str(e)}")



# Display the pivot table for the selected file
def view_pivot_table(filepath):
    try:
        # Load the selected table
        data = pd.read_json(filepath)  # Load the JSON file as a DataFrame

        # Ensure required columns are present
        if 'Date' not in data.columns or 'Component' not in data.columns or 'User_ID' not in data.columns:
            raise ValueError("The dataset does not contain the required columns: 'Date', 'Component', 'User_ID'.")

        # Add 'Month' column
        data['Date'] = pd.to_datetime(data['Date'])
        data['Month'] = data['Date'].dt.to_period('M')

        # Add a 'Count' column for aggregation purposes
        data['Count'] = 1

        # Create the pivot table
        pivot_table = data.pivot_table(
            index='User_ID',
            columns=['Component', 'Month'],
            values='Count',
            aggfunc='sum',
            fill_value=0
            
        )
        
        # Open a new window to display the pivot table
        pivot_window = tk.Toplevel()
        pivot_window.title("Pivot Table")
        pivot_window.geometry("1000x600")  # Adjusted window size

        # Add a frame to hold text and scrollbars
        frame = tk.Frame(pivot_window)
        frame.pack(fill=tk.BOTH, expand=True)

        # Add text widget with no wrapping
        pivot_text = tk.Text(frame, wrap=tk.NONE, height=30, width=80)
        pivot_text.insert(tk.END, "Pivot Table (User Interactions per Component per Month):\n\n")
        pivot_text.insert(tk.END, pivot_table.to_string())

        # Add horizontal and vertical scrollbars
        v_scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL, command=pivot_text.yview)
        h_scrollbar = tk.Scrollbar(frame, orient=tk.HORIZONTAL, command=pivot_text.xview)

        # Configure the text widget to work with scrollbars
        pivot_text.config(xscrollcommand=h_scrollbar.set, yscrollcommand=v_scrollbar.set)

        # Pack everything
        v_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)
        pivot_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Button to close the window
        tk.Button(pivot_window, text="Back to Table Manager", bg="#87CEEB", width=20, command=pivot_window.destroy).pack(pady=10)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while generating the pivot table: {str(e)}")
        

# Main window
root = tk.Tk()
root.title("User Activity File Management")
root.geometry("500x200")

btn_receive = tk.Button(root, text="Upload New Files", bg="#949494", fg="black", width=15, command=open_receive_window)
btn_view = tk.Button(root, text="View Processed Tables", bg="#949494", fg="black", width=18, command=open_table_manager)

btn_receive.pack(pady=(50, 10))
btn_view.pack(pady=(10, 20))

root.mainloop()
